In [55]:
#Import statements - DO NOT TOUCH
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm


from statsmodels.stats.proportion import proportions_ztest as ztest
from statsmodels.stats.weightstats import ttest_ind as ttest

# Dataset
working_df = pd.read_csv('sleep1.csv').drop(['Timestamp', 'Unnamed: 12', 'Unnamed: 13'], axis=1)

In [57]:
#First 10 entries of dataset
working_df.head(10)

,hours_of_sleep,gender,ap_classes,ec_hours,grade,ethnicity,Soc_econ_stat,num_siblings,el_12_effect,requirement,AD_Requirement
0,6,Male,4+,5-10 Hours,Senior,Asian,Medium,2,No Response,UNDER,UNDER
1,8,Prefer not to say,2-3,0-5 Hours,Sophomore,Asian,Medium,1,No Response,MET,MET
2,7,Male,4+,0-5 Hours,Sophomore,Asian,High,1,No Response,UNDER,MET
3,8,Female,2-3,0-5 Hours,Sophomore,Asian,High,1,No/negligible change,MET,MET
4,7,Prefer not to say,4+,0-5 Hours,Senior,Mixed,Medium,0,Increase,UNDER,MET
5,7,Female,4+,10-20 Hours,Senior,Prefer not to say,No Response,1,Increase,UNDER,MET
6,7,Prefer not to say,2-3,0-5 Hours,Junior,White,High,1,No/negligible change,UNDER,MET
7,7,Male,4+,5-10 Hours,Senior,Other,Medium,0,Increase,UNDER,MET
8,8,Male,2-3,10-20 Hours,Junior,White,High,2,Decrease,MET,MET
9,6,Female,4+,10-20 Hours,Junior,Asian,High,2,No/negligible change,UNDER,UNDER


# Data Summaries
Stat summaries - to view them, create a new box and type the name of the variable you want to see (eg. "summary_grade_sleep")

NOTE: IS CASE SENSITIVE

In [2]:
summary_gender_sleep = working_df.groupby('gender')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_grade_sleep = working_df.groupby('grade')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_ap_sleep = working_df.groupby('ap_classes')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_soc_sleep = working_df.groupby('Soc_econ_stat')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_sibling_sleep = working_df.groupby('num_siblings')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_el12_sleep = working_df.groupby('el_12_effect')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')
summary_ethnicity_sleep = working_df.groupby('ethnicity')['hours_of_sleep'].describe().drop('No Response', axis=0, errors='ignore')

In [54]:
summary_ethnicity_sleep

,count,mean,std,min,25%,50%,75%,max
ethnicity,,,,,,,,
Asian,37.0,6.054054,1.393356,4.0,5.00,6.0,7.00,8.0
Black/African American,1.0,7.000000,NaN,7.0,7.00,7.0,7.00,7.0
Latino/Latina/Latinx,1.0,7.000000,NaN,7.0,7.00,7.0,7.00,7.0
Mixed,9.0,6.000000,1.118034,5.0,5.00,6.0,7.00,8.0
Other,2.0,7.500000,0.707107,7.0,7.25,7.5,7.75,8.0
Prefer not to say,5.0,6.000000,1.000000,5.0,5.00,6.0,7.00,7.0
White,33.0,6.787879,0.857233,4.0,6.00,7.0,7.00,8.0


# SLEEP REQUIREMENTS

Assumptions: <br>
Teenager: 8-10 Hrs <br>
Adult: 7-8 Hrs

In [4]:
cross_gender_AD = pd.crosstab(working_df['gender'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_gender_reg = pd.crosstab(working_df['gender'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_grade_AD = pd.crosstab(working_df['grade'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_grade_reg = pd.crosstab(working_df['grade'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_ap_AD = pd.crosstab(working_df['ap_classes'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_ap_reg = pd.crosstab(working_df['ap_classes'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_ec_AD = pd.crosstab(working_df['ec_hours'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_ec_reg = pd.crosstab(working_df['ec_hours'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_sib_AD = pd.crosstab(working_df['num_siblings'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_sib_reg = pd.crosstab(working_df['num_siblings'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_econ_AD = pd.crosstab(working_df['Soc_econ_stat'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_econ_reg = pd.crosstab(working_df['Soc_econ_stat'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_ethnicity_AD = pd.crosstab(working_df['ethnicity'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_ethnicity_reg = pd.crosstab(working_df['ethnicity'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

cross_el12_AD = pd.crosstab(working_df['el_12_effect'], working_df['AD_Requirement']).drop('No Response', axis=0, errors='ignore')
cross_el12_reg = pd.crosstab(working_df['el_12_effect'], working_df['requirement']).drop('No Response', axis=0, errors='ignore')

# X^2 test for independence

Tested for people meeting and not meeting advised sleep

Output: <br>
Test Statistic <br>
P-value <br>
Degrees of freedom <br>
Expected frequency table <br>

## Gender

In [46]:
test_table = cross_gender_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 0.9952380952380951
p-value: 0.6079765008874592
degrees of freedom 2
[[19.09090909 15.90909091]
 [24.         20.        ]
 [ 4.90909091  4.09090909]]


## Grade Level

In [45]:
test_table = cross_grade_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 3.337132275132274
p-value: 0.3425078711816543
degrees of freedom 3
[[ 4.90909091  4.09090909]
 [ 7.63636364  6.36363636]
 [21.81818182 18.18181818]
 [13.63636364 11.36363636]]


## AP Class count

In [44]:
test_table = cross_ap_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 5.288137341585617
p-value: 0.07107151390310583
degrees of freedom 2
[[18.         15.        ]
 [14.18181818 11.81818182]
 [15.81818182 13.18181818]]


## Reported socioeconomic status

In [42]:
test_table = cross_econ_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 0.5812803421815049
p-value: 0.747784704167948
degrees of freedom 2
[[17.41772152 14.58227848]
 [ 1.63291139  1.36708861]
 [23.94936709 20.05063291]]


## Number of siblings

In [41]:
test_table = cross_sib_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 2.2945331281899595
p-value: 0.513569023452799
degrees of freedom 3
[[ 7.57647059  6.42352941]
 [26.51764706 22.48235294]
 [ 8.65882353  7.34117647]
 [ 3.24705882  2.75294118]]


## Thoughts on EL-12

In [43]:
test_table = cross_el12_AD
x2_val, pval, dof, freq = stats.chi2_contingency(test_table)

print(f"x2: {x2_val}")
print(f"p-value: {pval}")
print(f"degrees of freedom {dof}")
print(f"{freq}")

x2: 5.079435802292944
p-value: 0.0788886510500176
degrees of freedom 2
[[ 7.24050633  5.75949367]
 [11.69620253  9.30379747]
 [25.06329114 19.93670886]]


# T-Test
output format: (Test Statistic, P-value, Degrees of Freedom)

## Upperclassmen vs Lowerclassmen

In [49]:
upper_class_df = working_df[(working_df['grade']=='Senior') | (working_df['grade']=='Junior')]
lower_class_df = working_df[(working_df['grade']=='Freshman') | (working_df['grade']=='Sophomore')]
ttest(upper_class_df['hours_of_sleep'], lower_class_df['hours_of_sleep'])

(-2.314868540910588, 0.023002528892238312, 86.0)

## Extracurriculars [0-10 hr/wk vs 10-20 hr/wk]

In [48]:
lower_ec_df = working_df[(working_df['ec_hours']=='0-5 Hours') | (working_df['ec_hours']=='5-10 Hours')]
upper_ec_df = working_df[(working_df['ec_hours']=='10-20 Hours') | (working_df['ec_hours']=='20+ Hours')]
ttest(upper_ec_df['hours_of_sleep'], lower_ec_df['hours_of_sleep'])

(-0.5049405314097308, 0.6148921090669665, 86.0)

## Seniors vs Sophomores & Juniors

In [47]:
middle_class_df = working_df[(working_df['grade']=='Junior') | (working_df['grade']=='Sophomore')]
senior_df = working_df[(working_df['grade']=='Senior')]

ttest(middle_class_df['hours_of_sleep'], senior_df['hours_of_sleep'])

(2.779277974187426, 0.0068427546032571104, 77.0)

# Z test

output format: (Test statistic, P-value)

## Upperclassmen vs Lowerclassmen

In [14]:
u_ec = len(upper_class_df[(upper_class_df['ec_hours']=='10-20 Hours') | (upper_class_df['ec_hours']=='20+ Hours')])
l_ec = len(lower_class_df[(lower_class_df['ec_hours']=='10-20 Hours') | (lower_class_df['ec_hours']=='20+ Hours')])

count = np.array([u_ec, l_ec])
nobs = np.array([len(working_df), len(working_df)])

ztest(count, nobs)

(1.9092935402134767, 0.05622423516979415)

## AP Classes [4+ vs <4]

In [34]:
fourplus = len(working_df[(working_df['ap_classes']=='4+') & ((working_df['ec_hours']=='10-20 Hours') | (working_df['ec_hours']=='20+ Hours'))])
underfour = len(working_df[((working_df['ap_classes']=='2-3') | (working_df['ap_classes']=='0-1')) & 
                           ((working_df['ec_hours']=='10-20 Hours') | (working_df['ec_hours']=='20+ Hours'))])

count = np.array([fourplus, underfour])
nobs = np.array([len(working_df), len(working_df)])

ztest(count, nobs)

(-0.7637174160853909, 0.44503565978173254)